# Import Dependencies

In [1]:
import numpy as np
import pandas as pd

# Convert Multiple Excel sheets to CSV

In [5]:
excel_file = '../data/Moonshot Tracker Results - Auto.xlsx'
all_sheets = pd.read_excel(excel_file, sheet_name=None)
sheets = all_sheets.keys()

for sheet_name in sheets:
    sheet = pd.read_excel(excel_file, sheet_name=sheet_name)
    sheet.to_csv(f"../data/{sheet_name}.csv", index=False)

In [6]:
projects = pd.read_csv("../data/Projects.csv")
outputs = pd.read_csv("../data/Outputs.csv")

# Data Relationship

![image info](../data/data_relationship.jpeg)

In [12]:
projects.columns

Index(['Project ID', 'Project Title', 'Budget', 'Country', 'Country Code',
       'Link', 'Donors', 'VF or Non-VF', 'Output Count',
       'Direct Beneficiaries', 'GHG Emissions Reduction', 'description',
       'sdgs', 'solution', 'Gender Marker'],
      dtype='object')

In [16]:
project_target_column  = ['Project ID', 'Project Title', 'Budget', 'Country', 'Donors', 'VF or Non-VF', 'Output Count',
       'Direct Beneficiaries', 'description']

In [17]:
projects = projects[project_target_column]

In [18]:
outputs.columns

Index(['Project ID', 'Title', 'Link', 'Budget', 'Beneficiary Category',
       'Indicator', 'Baseline', 'Target', 'Notes', 'Donors',
       'Gender (% female)', 'VF or Non-VF', 'Tag', 'SEH Taxonomy',
       'RISE Taxonomy', 'Flagship', 'Technology', 'Output ID', 'Country Code',
       'Category', 'name 2', 'm49', 'continent-region', 'sub-region',
       'sids-region', 'un-member', 'undp-sids', 'un-region', 'Country Name',
       'Region', 'Economy', 'LDC', 'SIDS', 'LLDC', 'HDI', 'Status',
       'Direct Conversion Factor', 'Direct Beneficiaries', 'Output Category',
       'Beneficiary Category Pre', 'Description'],
      dtype='object')

In [20]:
outputs_target_column  = ['Project ID', 'Title','Beneficiary Category','Gender (% female)', 'continent-region', 'Country Name',
       'Region', 'Direct Beneficiaries', 'Output Category', 'Description']

In [21]:
outputs = outputs[outputs_target_column]

# Functions for Extracting Information
Our Goal: Generate text summaries of the UNDP portfolio for each country

Approach: 
1. Assign 'continent-region', 'Country Name', and 'Region' to the project dataframe.
2. Create a function to get each 'project' for the specified 'continent-region', 'Country Name', or 'Region'.
3. Create a function to get the 'output's of each 'project'.
4. Create a function to call the OpenAI API and generate a summary for the UNDP portfolio for each country using a suitable prompt.